In [6]:
import pandas as pd
import numpy as np
import folium
import folium.plugins
import geopandas as gpd

Important note: This dataset shows Electricity production by country. Electricity is not Energy!! A country might produce other types of energy such as gas, which is used to heat. This is not covered in this dataset.

## Import data and basic cleansing

In [7]:
df = pd.read_excel('Renewable_elec_production_percentage.xlsx', engine='openpyxl', sheet_name='Data')
df.head()

,CountryID,Country and area,1990,1991,1992,1993,1994,1995,1996,1997,...,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 30,Footnote
0,4,Afghanistan,67.730496,67.980296,67.99431,68.345324,68.704512,69.037037,70.37037,72.38806,...,85.986547,82.487562,85.90998,78.669276,85.319352,86.073501,86.152416,84.699454,NaN,NaN
1,8,Albania,87.613388,95.069296,95.889187,95.120551,96.617986,95.242411,96.625042,96.990741,...,99.987087,98.600901,100,100,100,100,100,100,NaN,NaN
2,12,Algeria,0.838301,1.689248,1.088264,1.818182,0.834674,0.979,0.629418,0.349016,...,0.380461,0.980009,1.08368,0.55101,0.39538,0.322684,0.251274,0.835328,NaN,NaN
3,20,Andorra,100,100,100,100,100,100,100,100,...,88.495575,86.813187,86.363636,88.695652,88.976378,86.868687,84.848485,83.962264,NaN,NaN
4,24,Angola,86.206897,82.655246,88.701162,93.684211,93.717277,93.75,89.980545,76.352531,...,67.957423,70.907804,60.799484,58.020935,53.175105,53.179724,56.123926,71.396586,NaN,NaN


In [8]:
# drop unnecessary rows and columns
df = df.drop(columns=['Unnamed: 30', 'Footnote'])
df = df.drop(df.index[224:252])
df = df.rename(columns = {"Country and area":"country"})
# replace '...' values with NaN
df = df.replace('...', np.NaN)
df

,CountryID,country,1990,1991,1992,1993,1994,1995,1996,1997,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,4,Afghanistan,67.730496,67.980296,67.994310,68.345324,68.704512,69.037037,70.370370,72.388060,...,68.654822,87.176603,85.986547,82.487562,85.909980,78.669276,85.319352,86.073501,86.152416,84.699454
1,8,Albania,87.613388,95.069296,95.889187,95.120551,96.617986,95.242411,96.625042,96.990741,...,98.139054,99.980883,99.987087,98.600901,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
2,12,Algeria,0.838301,1.689248,1.088264,1.818182,0.834674,0.979000,0.629418,0.349016,...,0.703350,0.794785,0.380461,0.980009,1.083680,0.551010,0.395380,0.322684,0.251274,0.835328
3,20,Andorra,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,79.746835,76.543210,88.495575,86.813187,86.363636,88.695652,88.976378,86.868687,84.848485,83.962264
4,24,Angola,86.206897,82.655246,88.701162,93.684211,93.717277,93.750000,89.980545,76.352531,...,75.409047,65.343189,67.957423,70.907804,60.799484,58.020935,53.175105,53.179724,56.123926,71.396586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,704,Viet Nam,61.856501,68.622560,74.117032,74.457809,74.512430,72.241944,70.868744,60.970134,...,35.406562,36.057710,29.457697,39.559692,45.148931,45.948674,44.059650,35.178661,36.007663,44.909546
220,876,Wallis and Futuna Is.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
221,887,Yemen,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.048333,0.056521,0.105758,0.117570,1.807130,10.637473,13.769752
222,894,Zambia,99.485266,99.485531,99.485861,99.460501,99.460501,99.331146,99.205132,99.307392,...,99.874135,99.878702,99.875574,99.869543,99.846390,99.849635,97.142857,96.986607,94.271056,85.992245


In [17]:
# Set up URL with shape data for all countries
url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

geoJSON_df = gpd.read_file(country_shapes)
geoJSON_df = geoJSON_df.rename(columns = {"name":"country"})
geoJSON_df.head()

,id,country,geometry
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6..."
2,ALB,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509..."
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407..."
4,ARG,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000..."


In [18]:
# replace manually the names in the GeoJSON dataframe with the names that are used in my dataset
geoJSON_df = geoJSON_df.replace('The Bahamas', 'Bahamas')
geoJSON_df = geoJSON_df.replace('Bolivia', 'Bolivia (Plur. State of)')
geoJSON_df = geoJSON_df.replace('Brunei', 'Brunei Darussalam')
geoJSON_df = geoJSON_df.replace('Central African Republic', 'Central African Rep.')
geoJSON_df = geoJSON_df.replace('Ivory Coast', 'Côte d\'Ivoire')
geoJSON_df = geoJSON_df.replace('Czech Republic', 'Czechia')
geoJSON_df = geoJSON_df.replace('Democratic Republic of the Congo', 'Dem. Rep. of the Congo')
geoJSON_df = geoJSON_df.replace('Falkland Islands', 'Falkland Is. (Malvinas)')
geoJSON_df = geoJSON_df.replace('Guinea Bissau', 'Guinea-Bissau')
geoJSON_df = geoJSON_df.replace('Iran', 'Iran (Islamic Rep. of)')
geoJSON_df = geoJSON_df.replace('South Korea', 'Korea, Republic of')
geoJSON_df = geoJSON_df.replace('North Korea', 'Korea, Dem.Ppl\'s.Rep.')
geoJSON_df = geoJSON_df.replace('Laos', 'Lao People\'s Dem. Rep.')
geoJSON_df = geoJSON_df.replace('Moldova', 'Republic of Moldova')
geoJSON_df = geoJSON_df.replace('Macedonia', 'North Macedonia')
geoJSON_df = geoJSON_df.replace('Russia', 'Russian Federation')
geoJSON_df = geoJSON_df.replace('Republic of Serbia', 'Serbia')
geoJSON_df = geoJSON_df.replace('Republic of the Congo', 'Congo')
geoJSON_df = geoJSON_df.replace('Swaziland', 'Eswatini')
geoJSON_df = geoJSON_df.replace('Syria', 'Syrian Arab Republic')
geoJSON_df = geoJSON_df.replace('East Timor', 'Timor-Leste')
geoJSON_df = geoJSON_df.replace('United Republic of Tanzania', 'United Rep. of Tanzania')
geoJSON_df = geoJSON_df.replace('United States of America', 'United States')
geoJSON_df = geoJSON_df.replace('Venezuela', 'Venezuela (Bolivar. Rep.)')
geoJSON_df = geoJSON_df.replace('Vietnam', 'Viet Nam')
geoJSON_df = geoJSON_df.replace('West Bank', 'State of Palestine')

# country geometries that have to get merged (data is not aggregated at such a low level in my dataset)
geoJSON_df = geoJSON_df.replace('Northern Cyprus', 'Cyprus')
geoJSON_df = geoJSON_df.replace('Somaliland', 'Somalia')
geoJSON_df = geoJSON_df.replace('Western Sahara', 'Morocco')
#geoJSON_df = geoJSON_df.dissolve(by='country', as_index=False)

In [19]:
# countries for which I have data available but not the country geometries (probably the country in included in the geometry of another country, e.g. French Guiana, Guadeloupe,... are included in France for the geometry but segregated in the electricity dataset)
# Decision: Leave these exceptions untreated! It would take way too much time to try to create or extract all of the missing geometries for these countries and they are mostly very small countries or islands, so the missing data is not visible on the final map
i=0
for index, row in df.iterrows():
    if not row['country'] in geoJSON_df['country'].values:
        print(row['country'])
        i+=1
print(i)

Andorra
Anguilla
Antigua and Barbuda
Aruba
Bahrain
Barbados
Bermuda
British Virgin Islands
Cabo Verde
Cayman Islands
China, Hong Kong SAR
China, Macao SAR
Comoros
Cook Islands
Curaçao
Dominica
Faeroe Islands
French Guiana
French Polynesia
Gibraltar
Grenada
Guadeloupe
Guam
Guernsey
Jersey
Kiribati
Liechtenstein
Maldives
Malta
Marshall Islands
Martinique
Mauritius
Mayotte
Micronesia (Fed. States of)
Montserrat
Nauru
Niue
Palau
Reunion
Samoa
Sao Tome and Principe
Serbia and Montenegro
Seychelles
Singapore
St. Helena and Depend.
St. Kitts-Nevis
St. Lucia
St. Pierre-Miquelon
St. Vincent-Grenadines
Tonga
Turks and Caicos Islands
Tuvalu
Wallis and Futuna Is.
53


In [ ]:
# countries for which I have the geometry, but not the data: This is fine
i=0
for index, row in geoJSON_df.iterrows():
    if not row['country'] in df['country'].values:
        print(row['country'])
        i+=1
print(i)

In [20]:
# merge country geometries with electricity data df
final_df = geoJSON_df.merge(df, on = "country")
final_df.head()

,id,country,geometry,CountryID,1990,1991,1992,1993,1994,1995,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,AFG,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066...",4,67.730496,67.980296,67.994310,68.345324,68.704512,69.037037,...,68.654822,87.176603,85.986547,82.487562,85.909980,78.669276,85.319352,86.073501,86.152416,84.699454
1,AGO,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6...",24,86.206897,82.655246,88.701162,93.684211,93.717277,93.750000,...,75.409047,65.343189,67.957423,70.907804,60.799484,58.020935,53.175105,53.179724,56.123926,71.396586
2,ALB,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509...",8,87.613388,95.069296,95.889187,95.120551,96.617986,95.242411,...,98.139054,99.980883,99.987087,98.600901,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
3,ARE,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407...",784,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.017887,0.048191,0.249725,0.228475,0.238433,0.588616
4,ARG,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000...",32,35.619856,30.400755,34.778668,39.048635,42.120086,40.226578,...,25.246254,28.060124,27.025973,24.581191,22.256984,24.063662,30.292095,28.937900,26.367071,29.030065


In [21]:
# filter data to 2017
data_2017 = final_df[['country', 'geometry', 2017]]
# round all values to two digits
data_2017 = data_2017.round({2017: 2})
data_2017.head()

,country,geometry,2017
0,Afghanistan,"POLYGON ((61.21082 35.65007, 62.23065 35.27066...",84.70
1,Angola,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6...",71.40
2,Albania,"POLYGON ((20.59025 41.85540, 20.46317 41.51509...",100.00
3,United Arab Emirates,"POLYGON ((51.57952 24.24550, 51.75744 24.29407...",0.59
4,Argentina,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000...",29.03


## Choropleth Map for 2017

In [53]:
m = folium.Map(location=[30,10], zoom_start = 2)

# add choropleth to map
folium.Choropleth(geo_data=data_2017, 
                  key_on='feature.properties.country',
                  data=data_2017, 
                  columns=['country', 2017], 
                  fill_color='RdYlGn',
                  nan_fill_color='white',
                  fill_opacity=0.8,
                  line_opacity=0.3,
                  legend_name="Renewable Electricity Production in 2017 (in %)",
                  bins=10).add_to(m)

style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1}

highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.3, 
                                'weight': 0.1}

hover = folium.features.GeoJson(
    data = data_2017, 
    control=False,
    highlight_function=highlight_function,
    style_function=style_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['country', '2017'],
        aliases=['Country','Renewable electricity production in %']
    )
)

m.add_child(hover)

m

## Timeslider Choropleth Map for 1990 - 2017

In [26]:
# colors from: https://colorbrewer2.org/#type=diverging&scheme=RdYlGn&n=10
from branca.colormap import linear
cmap = linear.RdYlGn_11.scale(0, 100)
cmap

In [43]:
# create timestamps from 1990 to 2017 (required format for TimeSliderChoropleth)
datetime_index = pd.date_range("1990", periods=28, freq="Y")
dt_index = datetime_index.astype(int) // 10 ** 9

Int64Index([ 662601600,  694137600,  725760000,  757296000,  788832000,
             820368000,  851990400,  883526400,  915062400,  946598400,
             978220800, 1009756800, 1041292800, 1072828800, 1104451200,
            1135987200, 1167523200, 1199059200, 1230681600, 1262217600,
            1293753600, 1325289600, 1356912000, 1388448000, 1419984000,
            1451520000, 1483142400, 1514678400],
           dtype='int64')

In [62]:
# Create style dictionary of structure: country_index: {timestamp: {color: ..., opacity: ...}}
styledata = {}

for idx, row in final_df.iterrows(): # iterate over every country
    # create a df for every country containing the color corresponding to percentage in the given year
    country_df = pd.DataFrame(columns=['color', 'opacity'])
    
    for i, v in enumerate(row[4:]): #iterating over every year
        # if no data available: white
        if np.isnan(v):
            country_df.loc[i,'color'] = '#ffffff'
        # otherwise get the color corresponding to percentage from cmap
        else:
            country_df.loc[i,'color'] = cmap(v)
        country_df.loc[i,'opacity'] = 0.8
        
    # replace [0,1,2,..] index with timestamps
    country_df = country_df.set_index(dt_index)
    styledata[idx] = country_df
    
# convert dataframe to dictionary
styledict = {
    country: data.to_dict(orient="index") for country, data in styledata.items()
}
styledict

{0: {662601600: {'color': '#b2de71ff', 'opacity': 0.8},
  694137600: {'color': '#b0de71ff', 'opacity': 0.8},
  725760000: {'color': '#b0de71ff', 'opacity': 0.8},
  757296000: {'color': '#afdd6fff', 'opacity': 0.8},
  788832000: {'color': '#addc6eff', 'opacity': 0.8},
  820368000: {'color': '#abdb6dff', 'opacity': 0.8},
  851990400: {'color': '#a4d86aff', 'opacity': 0.8},
  883526400: {'color': '#97d368ff', 'opacity': 0.8},
  915062400: {'color': '#8acd67ff', 'opacity': 0.8},
  946598400: {'color': '#8ecf67ff', 'opacity': 0.8},
  978220800: {'color': '#86cb66ff', 'opacity': 0.8},
  1009756800: {'color': '#94d168ff', 'opacity': 0.8},
  1041292800: {'color': '#6cc064ff', 'opacity': 0.8},
  1072828800: {'color': '#a5d96aff', 'opacity': 0.8},
  1104451200: {'color': '#a0d769ff', 'opacity': 0.8},
  1135987200: {'color': '#8cce67ff', 'opacity': 0.8},
  1167523200: {'color': '#a1d769ff', 'opacity': 0.8},
  1199059200: {'color': '#99d469ff', 'opacity': 0.8},
  1230681600: {'color': '#addc6eff',

In [46]:
# create geodataframe with country geometries (same index as in styledict)
countries_df = final_df[['geometry']]
countries_gdf = gpd.GeoDataFrame(countries_df)
countries_df.head()

,geometry
0,"POLYGON ((61.21082 35.65007, 62.23065 35.27066..."
1,"MULTIPOLYGON (((16.32653 -5.87747, 16.57318 -6..."
2,"POLYGON ((20.59025 41.85540, 20.46317 41.51509..."
3,"POLYGON ((51.57952 24.24550, 51.75744 24.29407..."
4,"MULTIPOLYGON (((-65.50000 -55.20000, -66.45000..."


In [60]:
# create map with time slider
tm = folium.Map(location=[20,10], zoom_start = 2)
folium.plugins.TimeSliderChoropleth(data=countries_gdf.to_json(), styledict=styledict).add_to(tm)

# add colormap
cmap.add_to(tm)
cmap.caption = "Renewable electricity production in %"

tm